In [1]:
import ipywidgets as widgets

In [21]:
class Section(widgets.VBox):
    def __init__(self, kind: str = 'Line'):
        width = '180px'
        self.kind = kind

        self.w_kind = widgets.Dropdown(
            value=self.kind,
            placeholder='Choose kind',
            options=['Line', 'Short', 'Tee'],
            description='Type',
            disabled=False,
            layout=widgets.Layout(width=width)
        )

        self.output = widgets.Output()

        # define the various widgets
        self.w_Dint = widgets.BoundedIntText(description='Dint [mm]', value=140, min=20, max=300, layout=widgets.Layout(width=width))
        self.w_Dout= widgets.BoundedIntText(description='Dout [mm]', value=230, min=100, max=400, layout=widgets.Layout(width=width))
        self.w_L = widgets.FloatSlider(description='L [mm]', min=0, max=10000, step=1, value=100, layout=widgets.Layout(width=width))
        self.w_sigma = widgets.Dropdown(
            value = 'Copper',
            options = ['Copper', 'Silver', 'Aluminium', 'Steel'],
            description = 'Metal',
            disabled = False,
            layout=widgets.Layout(width=width)
        )
        self.w_R = widgets.FloatText(description='Short Res. [Ohm]', value=1e-3, layout=widgets.Layout(width=width))

        super().__init__(children=self.UI, 
                         layout=widgets.Layout(border='solid'))

        self.w_kind.observe(self.on_kind_change, 'value')

    def on_kind_change(self, change):
        self.kind = change['new']
        self.__instantiate_ui()

    @property
    def UI(self):
        # instantiate the new kind widget 
        if self.kind == 'Line':
            UI = [self.w_kind, self.w_Dint, self.w_Dout, self.w_L, self.w_sigma]
        elif self.kind == 'Short':
            UI = [self.w_kind, self.w_Dint, self.w_Dout, self.w_R]
        elif self.kind == 'Tee':
            UI = [self.w_kind, self.w_Dint, self.w_Dout, self.w_sigma]
        return UI
    
    def __instantiate_ui(self):
        self.children = self.UI
        display(self)


In [22]:
class ResonatorBuilder(widgets.HBox):
    def __init__(self):
        # initial configuration
        self._config = [Section('Short'), Section('Line'), Section('Tee'), Section('Line'), Section('Short')]
        # define + and - buttons
        self.w_add_left = widgets.Button(icon="plus-square", layout=widgets.Layout(width='30px'))
        self.w_add_right = widgets.Button(icon="plus-square", layout=widgets.Layout(width='30px'))
        self.w_del_left = widgets.Button(icon="minus-square", layout=widgets.Layout(width='30px'))
        self.w_del_right = widgets.Button(icon="minus-square", layout=widgets.Layout(width='30px'))
        # define callbacks for + and - buttons
        self.w_add_left.on_click(self.add_section_to_the_left)
        self.w_add_right.on_click(self.add_section_to_the_right)
        self.w_del_left.on_click(self.del_section_to_the_left)
        self.w_del_right.on_click(self.del_section_to_the_right)
        
        self.output = widgets.Output()

        super().__init__(children=self.UI)
        
    @property
    def UI(self):
        return [
            widgets.VBox([self.w_add_left, self.w_del_left]), 
            *self.config, 
            widgets.VBox([self.w_add_right, self.w_del_right]), 
            self.output
        ]
        
    def add_section_to_the_left(self, change):
        self.config.insert(0, Section())
        self.__update_display()

    def add_section_to_the_right(self, change):
        self.config.append(Section())
        self.__update_display()
            
    def del_section_to_the_left(self, change):
        self.config.pop(0)
        self.__update_display()
            
    def del_section_to_the_right(self, change):
        self.config.pop()
        self.__update_display()
            
    def __update_display(self):
        self.children = self.UI
        display(self)

    @property
    def config(self):
        return self._config

    @config.setter
    def config(self, cfg):
        self._config = cfg 

In [23]:
rb=ResonatorBuilder()
rb

ResonatorBuilder(children=(VBox(children=(Button(icon='plus-square', layout=Layout(width='30px'), style=Button…

In [20]:
rb.config

[Section(children=(Dropdown(description='Section Type', layout=Layout(width='180px'), options=('Line', 'Short', 'Tee'), value='Line'), BoundedIntText(value=140, description='Dint [mm]', layout=Layout(width='180px'), max=300, min=20), BoundedIntText(value=230, description='Dout [mm]', layout=Layout(width='180px'), max=400, min=100), FloatSlider(value=100.0, description='Length [mm]', layout=Layout(width='180px'), max=10000.0, step=1.0), Dropdown(description='Metal', layout=Layout(width='180px'), options=('Copper', 'Silver', 'Aluminium', 'Steel'), value='Copper')), layout=Layout(border='solid')),
 Section(children=(Dropdown(description='Section Type', index=1, layout=Layout(width='180px'), options=('Line', 'Short', 'Tee'), value='Short'), BoundedIntText(value=140, description='Dint [mm]', layout=Layout(width='180px'), max=300, min=20), BoundedIntText(value=230, description='Dout [mm]', layout=Layout(width='180px'), max=400, min=100), FloatText(value=0.001, description='Short Res. [Ohm]',